Usually, we might occur with three types of bad run number errors from AMS raw data as following pic, 
1) repeated in the same file;     
2) run number is not increased with time;     
3) repeated number existed in different files.


Check and revise the run number in raw data files one by one in New HDF browser might not be a good choice if a large number of files need to be processed.

[![enter image description here][1]][1]



  [1]: https://i.stack.imgur.com/V1vkn.png

In [12]:
import numpy as np
import pandas as pd
import os,shutil,h5py

In [13]:
# 1. Traverse the work folder for orignial run number list
path  = "./"  # set the path of the folder which contains all H5 files needed to be analysis together.
files = os.listdir(path)
run_number,run_file = [],[]
for file in files:
    if file.endswith(".h5"):
        hf = h5py.File(path+file, 'r') 
        run_number+=list(np.array(hf.get('RunInfo'))[:,0].astype(int))
        run_file  +=[file for i in range(0,(hf.get('RunInfo')).shape[0],1)] 
df_run = pd.DataFrame() # create the dataframe with the information of run number and corresponding filename. 
df_run['Run number'],df_run['file'] = run_number, run_file

In [20]:
df_run

,Run number,file
0,459630,210416_459630_m.h5
1,459631,210416_459630_m.h5
2,459632,210416_459632_m.h5
3,459633,210416_459632_m.h5
4,459634,210416_459632_m.h5
5,459635,210416_459632_m.h5
6,459636,210416_459636_m.h5
7,459637,210416_459637_m.h5
8,459638,210416_459637_m.h5
9,459639,210416_459637_m.h5


In [10]:
# 2. Check the distribution of problematic run number 
# 2.1 repeated run number
# locate the problematic files that can be double-checked via New HDF browser in Igor
df_run[df_run['Run number'].duplicated()]


,Run number,file
2,459631,210416_459631_m.h5
5,459631,210416_459631_m.h5
6,459632,210416_459632_m.h5
7,459633,210416_459633_m.h5


In [11]:
# 2.2 run number is not strictly increasing, i.e., not increase with time
df_run[~(sorted(df_run["Run number"])==df_run["Run number"])] 

,Run number,file
3,459632,210416_459631_m.h5
4,459633,210416_459631_m.h5
5,459631,210416_459631_m.h5
6,459632,210416_459632_m.h5


In [16]:
# 3. Creat new consecutive run number list 
os.mkdir(path+'/output') # make a new path to save the revised files
df_run.to_excel(path+'/output/'+"old_run_name_list.xls", index= False) # save the original information of run number and filename
start_ = df_run['Run number'].iloc[0]
df_run_new = df_run 
df_run_new["Run number"] = np.arange(start_, start_+len(df_run),1) # simply create the run number list starting from the first run number of the first file.

In [18]:
# 4. Rewrite the run number to hdf5 files 
files = os.listdir(path)
run_file = []
for file in files:
    if file.endswith(".h5"):
        hf = h5py.File(path+file, 'r+') 
        hf['RunInfo'][:,0] = list(df_run_new[df_run_new["file"]==file]['Run number']) # assign new value to the run number list for each file.
        new_start  =  hf['RunInfo'][:,0][0]
        new_file   = file.split('_')[0]+'_'+str(int(new_start))+"_"+file.split('_')[2] # creat the new list to re-name files, i.e., make the revised first run number shown in the filename.
        run_file  +=[new_file for i in range(0,(hf.get('RunInfo')).shape[0],1)]
        hf.close()
        shutil.copy2(path+file,path+"/output/"+new_file)

df_run_new["file"] = run_file
df_run_new.to_excel(path+'/output/'+"new_run_name_list.xls", index= False) # save the revised information of run number and filename.


In [19]:
df_run_new

,Run number,file
0,459630,210416_459630_m.h5
1,459631,210416_459630_m.h5
2,459632,210416_459632_m.h5
3,459633,210416_459632_m.h5
4,459634,210416_459632_m.h5
5,459635,210416_459632_m.h5
6,459636,210416_459636_m.h5
7,459637,210416_459637_m.h5
8,459638,210416_459637_m.h5
9,459639,210416_459637_m.h5
